In [1]:
%load_ext autoreload
%autoreload 2

In [104]:
model_dir = '/kuacc/users/mali18/dicomogan/logs/_separate-cond-larger-n-frames-relase-const2022-10-05T00-45-44'
img_root = '/kuacc/users/abond19/datasets/aligned_fashion_dataset'
inverted_img_root =  '/kuacc/users/abond19/datasets/inverted_fashion_dataset'
inversion_root =  '/kuacc/users/abond19/datasets/w+_fashion_dataset/fashion/PTI/'

In [77]:
device = 'cuda'

In [78]:
tgt_desc = [
    'Blue T-shirts Made from rayon-nylon blend Round neckline Slogan printed front Pearl detail Short sleeves Regular fit',
    'Dresses Shift White One shoulder design Made from cotton Lace detailed ruffled yoke Regular fit Mini length Sleeveless',
    'Multi Leggings Made from poly-lycra blend Elasticated waistband All over crane print Tape detail at sides Bodycon fit'
]

In [118]:
videos = [
#     '8c110571',
    '2c111960',
    '8c110147'
]

In [119]:
from experiments_utils import *
model = load_model_from_dir(model_dir).to(device)

Restored from /kuacc/users/mali18/dicomogan/logs/_separate-cond-larger-n-frames-relase-const2022-10-05T00-45-44/VideoManipulation/_separate-cond-larger-n-frames-relase-const2022-10-05T00-45-44/checkpoints/epoch=399-step=299599.ckpt


In [121]:
model = model.to(device)

In [47]:
def save_gif(video, save_path):
    # Assuming that the current shape is T x C x H x W
    import imageio
    with imageio.get_writer(f'{save_path}.gif', mode='I') as writer:
        for img in video:
            writer.append_data(img)

In [33]:
# load images
import os
import torch 
from PIL import Image
import torchvision.transforms as transforms
IMG_EXTENSIONS = ['.png', '.PNG']
TXT_EXTENSIONS = ['.txt']

crop = None
size = (256, 192)
trans_list = []
if crop is not None:
    trans_list.append(transforms.CenterCrop(tuple(crop)))
if size is not None:
    trans_list.append(transforms.Resize(tuple(size)))
trans_list.append(transforms.ToTensor())
img_transform=transforms.Compose(trans_list)

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def is_text_file(filename):
    return any(filename.endswith(extension) for extension in TXT_EXTENSIONS)

def get_image(img_path):
    img = Image.open(img_path).convert('RGB')
    return img

def get_inversion(inversion_path):
    w_vector = torch.load(inversion_path, map_location='cpu')
    assert (w_vector.shape == (1, 18, 512)), "Inverted vector has incorrect shape"
    return w_vector

def load_video(vid_path):
    images, inversions, sampleT, inversion_imgs = [], [], [], [] 
    fname = vid_path
    for f in sorted(os.listdir(os.path.join(img_root, fname)))[5:25]:
        if is_image_file(f):
            imname = f[:-4]
            images.append(img_transform(get_image(os.path.join(img_root, fname, f))))
            inversion_imgs.append(img_transform(get_image(os.path.join(inverted_img_root, fname, f))))
            inversions.append(get_inversion(os.path.join(os.path.join(inversion_root, fname, imname + ".pt"))))
            sampleT.append(int(imname))
    
    return torch.stack(images).to(device), torch.cat(inversions, 0).to(device), torch.Tensor(sampleT).to(device), torch.stack(inversion_imgs).to(device)

In [34]:
from torch import nn
def forward_w_color(self, videos, inversions, sampleT, input_desc):
    """
    return a dictionary of tensors in the range [-1, 1]
    """
    ret = dict()

    vid = videos # B x T x ch x H x W -- range [0, 1]
    input_desc = input_desc 
    sampleT = sampleT 
    
    bs, T, ch, height, width = vid.size()
    n_frames = T
    ts = (sampleT)*0.01
    ts = ts - ts[0] 


    video_sample = vid # B x T x C x H x W 
    video_sample = video_sample.permute(1,0,2,3,4) # T x B x C x H x W 
    video_sample = video_sample.contiguous().view(n_frames * bs, ch, height, width) # T*B x C x H x W 
    video_sample_norm = video_sample * 2 - 1 # range [-1, 1] to pass to the generator and disc

    # inversions reshape
    inversions_bf = inversions # B, T x n_layers x D
    bs, T, n_channels, dim = inversions_bf.shape
    inversions_tf = inversions_bf.permute(1, 0, 2, 3)
    inversions = inversions_tf.contiguous().reshape(T * bs, n_channels, dim) # T * B x n_layers x D

    # downsample res for vae
    vid_rs_full = nn.functional.interpolate(video_sample, scale_factor=0.5, mode="bicubic", align_corners=False, recompute_scale_factor=True)
    vid_rs = vid_rs_full.view(n_frames, bs, ch, int(height*0.5),int(width*0.5) )
    vid_rs = vid_rs.permute(1,0,2,3,4) #  B x T x C x H//2 x W//2

    # encode text
    txt_feat = self.clip_encode_text(input_desc)  # B x D
    txt_feat = txt_feat.unsqueeze(0).repeat(n_frames,1,1)
    txt_feat = txt_feat.view(bs * n_frames, -1)  # T*B x D

    # vae encode frames
    zs, zd, mu_logvar_s, mu_logvar_d = self.bVAE_enc(vid_rs, ts)
    z_vid = torch.cat((zs, zd), 1) # T*B x D 

    muT, logvarT = self.text_enc(txt_feat)
    zT = self.reparametrize(muT, logvarT) # T*B x D 

    # generate with mathching text
    latentw = self.mapping(z_vid[:,self.vae_cond_dim:])
    
    frame_rep = torch.cat((latentw, txt_feat), -1) # T*B x D1+D2

    # predict latents delta
    w_latents = inversions + self.delta_inversion_weight * self.style_mapper(inversions, frame_rep)
    ret = self.stylegan_G(w_latents) / 2 + 0.5 
    ret = ret.reshape(T, bs, ret.shape[1], ret.shape[2], ret.shape[3]).permute(1, 0, 2, 3, 4)
    return ret



In [35]:
from torch import nn
def forward_from_mean(self, videos, inversions, sampleT, input_desc):
    """
    return a dictionary of tensors in the range [-1, 1]
    """
    ret = dict()

    vid = videos # B x T x ch x H x W -- range [0, 1]
    input_desc = input_desc 
    sampleT = sampleT 
    
    bs, T, ch, height, width = vid.size()
    n_frames = T
    ts = (sampleT)*0.01
    ts = ts - ts[0] 


    video_sample = vid # B x T x C x H x W 
    video_sample = video_sample.permute(1,0,2,3,4) # T x B x C x H x W 
    video_sample = video_sample.contiguous().view(n_frames * bs, ch, height, width) # T*B x C x H x W 
    video_sample_norm = video_sample * 2 - 1 # range [-1, 1] to pass to the generator and disc

    # inversions reshape
    inversions_bf = inversions # B, T x n_layers x D
    bs, T, n_channels, dim = inversions_bf.shape
    inversions_tf = inversions_bf.permute(1, 0, 2, 3)
    inversions = inversions_tf.contiguous().reshape(T * bs, n_channels, dim) # T * B x n_layers x D

    # downsample res for vae
    vid_rs_full = nn.functional.interpolate(video_sample, scale_factor=0.5, mode="bicubic", align_corners=False, recompute_scale_factor=True)
    vid_rs = vid_rs_full.view(n_frames, bs, ch, int(height*0.5),int(width*0.5) )
    vid_rs = vid_rs.permute(1,0,2,3,4) #  B x T x C x H//2 x W//2

    # encode text
    txt_feat = self.clip_encode_text(input_desc)  # B x D
    txt_feat = txt_feat.unsqueeze(0).repeat(n_frames,1,1)
    txt_feat = txt_feat.view(bs * n_frames, -1)  # T*B x D

    # vae encode frames
    zs, zd, mu_logvar_s, mu_logvar_d = self.bVAE_enc(vid_rs, ts)
    z_vid = torch.cat((zs, zd), 1) # T*B x D 

    muT, logvarT = self.text_enc(txt_feat)
    zT = self.reparametrize(muT, logvarT) # T*B x D 

    # generate with mathching text
    latentw = self.mapping(z_vid[:,self.vae_cond_dim:])

    frame_rep = torch.cat((latentw, txt_feat), -1) # T*B x D1+D2

    # predict latents delta
    src_inversion = inversions_tf.mean(0, keepdims=True) # 1 x B x 18 x 512
    src_inversion_tf = src_inversion.repeat(T, 1, 1, 1)
    src_inversion = src_inversion_tf.reshape(T*bs, n_channels, dim)
    w_latents = src_inversion + self.delta_inversion_weight * self.style_mapper(src_inversion, frame_rep)
    ret = self.stylegan_G(w_latents) / 2 + 0.5 
    ret = ret.reshape(T, bs, ret.shape[1], ret.shape[2], ret.shape[3]).permute(1, 0, 2, 3, 4)
    return ret



In [110]:
from torch import nn
def tmp_forward(self, videos, inversions, sampleT, input_desc):
    """
    return a dictionary of tensors in the range [-1, 1]
    """
    ret = dict()

    vid = videos # B x T x ch x H x W -- range [0, 1]
    input_desc = input_desc 
    sampleT = sampleT 
    
    bs, T, ch, height, width = vid.size()
    n_frames = T
    ts = (sampleT)*0.01
    ts = ts - ts[0] 


    video_sample = vid # B x T x C x H x W 
    video_sample = video_sample.permute(1,0,2,3,4) # T x B x C x H x W 
    video_sample = video_sample.contiguous().view(n_frames * bs, ch, height, width) # T*B x C x H x W 
    video_sample_norm = video_sample * 2 - 1 # range [-1, 1] to pass to the generator and disc

    # inversions reshape
    inversions_bf = inversions # B, T x n_layers x D
    bs, T, n_channels, dim = inversions_bf.shape
    inversions_tf = inversions_bf.permute(1, 0, 2, 3)
    inversions = inversions_tf.contiguous().reshape(T * bs, n_channels, dim) # T * B x n_layers x D

    # downsample res for vae
    vid_rs_full = nn.functional.interpolate(video_sample, scale_factor=0.5, mode="bicubic", align_corners=False, recompute_scale_factor=True)
    vid_rs = vid_rs_full.view(n_frames, bs, ch, int(height*0.5),int(width*0.5) )
    vid_rs = vid_rs.permute(1,0,2,3,4) #  B x T x C x H//2 x W//2

    # encode text
    txt_feat = self.clip_encode_text(input_desc)  # B x D
    txt_feat = txt_feat.unsqueeze(0).repeat(n_frames,1,1)
    txt_feat = txt_feat.view(bs * n_frames, -1)  # T*B x D

    # vae encode frames
    zs, zd, mu_logvar_s, mu_logvar_d = self.bVAE_enc(vid_rs, ts)
    
    # swap dynamics
    zd = zd.view(T, bs, -1).contiguous()
    zd = torch.roll(zd, 1, 1)
    zd = zd.view(T*bs, -1).contiguous()
    
    
    z_vid = torch.cat((zs, zd), 1) # T*B x D 

    muT, logvarT = self.text_enc(txt_feat)
    zT = self.reparametrize(muT, logvarT) # T*B x D 

    # generate with mathching text
    latentw = self.mapping(z_vid[:,self.vae_cond_dim:])

    frame_rep = (latentw, txt_feat) # T*B x D1+D2

    # predict latents delta
    src_inversion = inversions_tf.mean(0, keepdims=True) # 1 x B x 18 x 512
#     src_inversion = inversions_tf[-1:]# 1 x B x 18 x 512
    src_inversion_tf = src_inversion.repeat(T, 1, 1, 1)
#     src_inversion_tf = inversions_tf
    src_inversion = src_inversion_tf.reshape(T*bs, n_channels, dim)
    w_latents = src_inversion + self.delta_inversion_weight * self.style_mapper(src_inversion, *frame_rep)
    ret = self.stylegan_G(w_latents) / 2 + 0.5 
    ret = ret.reshape(T, bs, ret.shape[1], ret.shape[2], ret.shape[3]).permute(1, 0, 2, 3, 4)
    return ret


In [72]:
def forward_hairclip(self, videos, inversions, sampleT, input_desc):
    """
    return a dictionary of tensors in the range [-1, 1]
    """
    ret = dict()

    vid = videos # B x T x ch x H x W -- range [0, 1]
    input_desc = input_desc 
    sampleT = sampleT 

    bs, T, ch, height, width = vid.size()
    n_frames = T
    ts = (sampleT)*0.01
    ts = ts - ts[0] 


    video_sample = vid # B x T x C x H x W 
    video_sample = video_sample.permute(1,0,2,3,4) # T x B x C x H x W 
    video_sample = video_sample.contiguous().view(n_frames * bs, ch, height, width) # T*B x C x H x W 
    video_sample_norm = video_sample * 2 - 1 # range [-1, 1] to pass to the generator and disc

    # inversions reshape
    inversions_bf = inversions # B, T x n_layers x D
    bs, T, n_channels, dim = inversions_bf.shape
    inversions_tf = inversions_bf.permute(1, 0, 2, 3)
    inversions = inversions_tf.contiguous().reshape(T * bs, n_channels, dim) # T * B x n_layers x D

    txt_feat = self.get_text_embedding(input_desc) # B x D
    txt_feat = txt_feat.unsqueeze(0).repeat(n_frames, 1, 1) # T x B x D
    txt_feat = txt_feat.view(bs * n_frames, -1) # T * B x D

    adjusted_latent = inversions + self.delta_inversion_weight * self.mapping_network(inversions, txt_feat)

    ret = self.G(adjusted_latent) / 2 + 0.5 
    ret = ret.reshape(T, bs, ret.shape[1], ret.shape[2], ret.shape[3]).permute(1, 0, 2, 3, 4)
    return ret


In [193]:
def forward_hairclip_same_dir(self, videos, inversions, sampleT, input_desc):
    """
    return a dictionary of tensors in the range [-1, 1]
    """
    ret = dict()

    vid = videos # B x T x ch x H x W -- range [0, 1]
    input_desc = input_desc 
    sampleT = sampleT 

    bs, T, ch, height, width = vid.size()
    n_frames = T
    ts = (sampleT)*0.01
    ts = ts - ts[0] 


    video_sample = vid # B x T x C x H x W 
    video_sample = video_sample.permute(1,0,2,3,4) # T x B x C x H x W 
    video_sample = video_sample.contiguous().view(n_frames * bs, ch, height, width) # T*B x C x H x W 
    video_sample_norm = video_sample * 2 - 1 # range [-1, 1] to pass to the generator and disc

    # inversions reshape
    inversions_bf = inversions # B, T x n_layers x D
    bs, T, n_channels, dim = inversions_bf.shape
    inversions_tf = inversions_bf.permute(1, 0, 2, 3)
    inversions = inversions_tf.contiguous().reshape(T * bs, n_channels, dim) # T * B x n_layers x D

    txt_feat = self.get_text_embedding(input_desc) # B x D
    txt_feat = txt_feat.unsqueeze(0).repeat(n_frames, 1, 1) # T x B x D
    txt_feat = txt_feat.view(bs * n_frames, -1) # T * B x D
    
    deltas = self.delta_inversion_weight * self.mapping_network(inversions, txt_feat) # T * B x n_layers x D
    deltas = deltas.reshape(T, bs, n_channels, dim)[0:1]
    deltas = deltas.repeat(T, 1, 1, 1).reshape(T * bs, n_channels, dim)
    adjusted_latent = inversions + deltas

    ret = self.G(adjusted_latent) / 2 + 0.5 
    ret = ret.reshape(T, bs, ret.shape[1], ret.shape[2], ret.shape[3]).permute(1, 0, 2, 3, 4)
    return ret

In [98]:
def forward_triple(self, videos, inversions, sampleT, input_desc):
        """
        return a dictionary of tensors in the range [-1, 1]
        """
        vid = videos # B x T x ch x H x W -- range [0, 1]
        input_desc = input_desc 
        sampleT = sampleT 
        
        bs, T, ch, height, width = vid.size()
        n_frames = T
        ts = (sampleT)*0.01
        ts = ts - ts[0] 


        video_sample = vid # B x T x C x H x W 
        video_sample = video_sample.permute(1,0,2,3,4) # T x B x C x H x W 
        video_sample = video_sample.contiguous().view(n_frames * bs, ch, height, width) # T*B x C x H x W 
        video_sample_norm = video_sample * 2 - 1 # range [-1, 1] to pass to the generator and disc

        # inversions reshape
        inversions_bf = inversions # B, T x n_layers x D
        bs, T, n_channels, dim = inversions_bf.shape
        inversions_tf = inversions_bf.permute(1, 0, 2, 3)
        inversions = inversions_tf.contiguous().reshape(T * bs, n_channels, dim) # T * B x n_layers x D

        # downsample res for vae
        vid_rs_full = nn.functional.interpolate(video_sample, scale_factor=0.5, mode="bicubic", align_corners=False, recompute_scale_factor=True)
        vid_rs = vid_rs_full.view(n_frames, bs, ch, int(height*0.5),int(width*0.5) )
        vid_rs = vid_rs.permute(1,0,2,3,4) #  B x T x C x H//2 x W//2

        # encode text
        txt_feat = self.clip_encode_text(input_desc)  # B x D
        txt_feat = txt_feat.unsqueeze(0).repeat(n_frames,1,1)
        txt_feat = txt_feat.view(bs * n_frames, -1)  # T*B x D

        # vae encode frames
        zs, zd, mu_logvar_s, mu_logvar_d = self.bVAE_enc(vid_rs, ts)
        z_vid = torch.cat((zs, zd), 1) # T*B x D 
        video_style = zs[:, :self.vae_cond_dim]
        video_content = zs[:, self.vae_cond_dim:]
        video_dynamics = zd

        muT, logvarT = self.text_enc(txt_feat)
        zT = self.reparametrize(muT, logvarT) # T*B x D 
        

        # roll batch-wise
        
        # frame_rep = (latentw, video_style) # T*B x D1+D2
        # frame_rep_txt_mismatched = (latentw, text_video_style_mismatch) # T*B x D1+D2
        
        # frame rep (video_style, video_content, dynamics)
        video_dynamics = torch.roll(video_dynamics, 1, dims=0)
        
        frame_rep = (txt_feat, video_content, video_dynamics) # T*B x D1+D2

        # predict latents delta
        inversions_tf = self.inversion_mapper(inversions_tf)
        src_inversion = inversions_tf.mean(0, keepdims=True) # 1 x B x 18 x 512
        src_inversion_tf = src_inversion.repeat(T, 1, 1, 1)
#         src_inversion_tf = torch.cat([inversions_tf[:, :, :4, :], src_inversion_tf[:, :, 4:, :]], -2)
        src_inversion = src_inversion_tf.reshape(T*bs, n_channels, dim)
        w_latents = src_inversion + self.delta_inversion_weight * self.style_mapper(src_inversion, *frame_rep)

        ret = self.stylegan_G(w_latents) / 2 + 0.5 
        ret = ret.reshape(T, bs, ret.shape[1], ret.shape[2], ret.shape[3]).permute(1, 0, 2, 3, 4)
        return ret

In [85]:
from tqdm import tqdm
with torch.no_grad():
    for video in tqdm(videos):
        images, inversions, sampleT, inversion_imgs = load_video(video)
        
        save_dir = os.path.join('results', f"{model_dir.split('/')[-1]}_fixed", video)
        os.makedirs(save_dir, exist_ok=True)
        # org
        save_gif(images.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/original')
        
        # inversion
        save_gif(inversion_imgs.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/inversion')
        
        for j, des in enumerate(tgt_desc):
#         images = images.unsqueeze(0).repeat(ln_desc, 1, 1, 1, 1)
#         inversions = inversions.unsqueeze(0).repeat(ln_desc, 1, 1, 1)
            edited_videos = forward_triple(model, images.unsqueeze(0), inversions.unsqueeze(0), sampleT, [des])[0]
            save_gif(edited_videos.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/desc_{j}')
        
        
        

  0%|          | 0/3 [00:00<?, ?it/s]Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conve

Lossy conversion from float32 to uint8. Range [-0.024552345275878906, 1.0129461288452148]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.017495334148406982, 1.0122778415679932]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.022822022438049316, 1.0125056505203247]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.03937453031539917, 1.0137420892715454]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.0260542631149292, 1.0131852626800537]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.012782096862792969, 1.0127665996551514]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.03488719463348389

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

Lossy conversion from float32 to uint8. Range [-0.11134040355682373, 1.0237209796905518]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.07465499639511108, 1.0165412425994873]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.06280040740966797, 1.0169250965118408]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.05125772953033447, 1.017349362373352]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.059805333614349365, 1.0153660774230957]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.05131256580352783, 1.0164079666137695]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.06150084733963013, 1

Lossy conversion from float32 to uint8. Range [0.06281805038452148, 1.0177063941955566]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.051387935876846313, 1.0172444581985474]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.06680458784103394, 1.0170516967773438]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.054022133350372314, 1.01735520362854]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.062427639961242676, 1.0358856916427612]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.06365248560905457, 1.0447471141815186]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.06135272979736328, 1.02200

Lossy conversion from float32 to uint8. Range [0.055663466453552246, 1.0196425914764404]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.06445547938346863, 1.0176777839660645]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.052170753479003906, 1.0200176239013672]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.05147567391395569, 1.0176637172698975]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.05489689111709595, 1.018641471862793]. Convert image to uint8 prior to saving to suppress this warning.
100%|██████████| 3/3 [00:52<00:00, 17.50s/it]


In [122]:
# swape the dynamics
from tqdm import tqdm
with torch.no_grad():
    for j, des in enumerate(tgt_desc):
        a_images, a_inv, a_sampleT = [], [], []
        for video in tqdm(videos):
            images, inversions, sampleT, inversion_imgs = load_video(video)
            save_dir = os.path.join('results', f"{model_dir.split('/')[-1]}_swapped", video)
            os.makedirs(save_dir, exist_ok=True)
            # org
#             save_gif(images.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/original')

            # inversion
#             save_gif(inversion_imgs.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/inversion')
            
            a_images.append(images)
            a_inv.append(inversions)
            a_sampleT.append(sampleT)

        

#         images = images.unsqueeze(0).repeat(ln_desc, 1, 1, 1, 1)
#         inversions = inversions.unsqueeze(0).repeat(ln_desc, 1, 1, 1)
        edited_videos = tmp_forward(model, torch.stack(a_images), torch.stack(a_inv), sampleT, [des] * len(videos))
        
        for edited_video, video in zip(edited_videos, videos):
            save_dir = os.path.join('results', f"{model_dir.split('/')[-1]}_swapped", video)
            save_gif(edited_video.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/desc_{j}')
        

100%|██████████| 2/2 [00:02<00:00,  1.49s/it]
Lossy conversion from float32 to uint8. Range [-0.19631195068359375, 1.017393946647644]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.20281076431274414, 1.0173282623291016]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.22139930725097656, 1.017282247543335]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.22032618522644043, 1.0172398090362549]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.22554844617843628, 1.0171841382980347]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.2322547435760498, 1.0171765089035034]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float

Lossy conversion from float32 to uint8. Range [-0.09279298782348633, 1.0246132612228394]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.09672820568084717, 1.0245598554611206]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.094204843044281, 1.0245387554168701]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.09099411964416504, 1.0244719982147217]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.09268641471862793, 1.024476408958435]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.09293770790100098, 1.0244126319885254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.09410834312438965, 1.02

Lossy conversion from float32 to uint8. Range [-0.04713314771652222, 1.015479564666748]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.047874510288238525, 1.015477180480957]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.040570855140686035, 1.022615671157837]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.0497434139251709, 1.0260720252990723]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.046389758586883545, 1.0259310007095337]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.042775750160217285, 1.0443787574768066]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.04470956325531006, 1

In [59]:
model.inversion_mapper[0].weight.norm()

tensor(22.5453, device='cuda:0', grad_fn=<CopyBackwards>)

In [61]:
model.inversion_mapper[0].bias.sum()

tensor(-1.2546, device='cuda:0', grad_fn=<SumBackward0>)

In [86]:
inversions.shape

torch.Size([20, 18, 512])

In [87]:
l2_latent_loss = nn.MSELoss()

In [96]:
l2_latent_loss(inversions.mean(0), inversions[10])

tensor(0.0041, device='cuda:0')